#### [ 임베딩 이해 ]

In [1]:
## => Embedding  : 고차원 피쳐 ==> 저차원 
## => Pytorch    : Embedding Layer 제공 
##    * 고정 길이 : Embedding    Layer
##    * 동적 길이 : EmbeddingBag Layer
## => 사전 학습된 성능 좋은 Embeddign Model들 존재
##    : Word2Vec, Doc2Vec, FastText, ..., CBOW, Skip-gram...

In [2]:
## 모듈 로딩
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
## 임의의 데이터 => 토큰화 
sentence = "Repeat is the best medicine for memory".split()

In [4]:
## 데이터기반의 단어사전 생성 [1] 단어 추출
vocab = list(set(sentence))
print(vocab)

['for', 'Repeat', 'medicine', 'is', 'the', 'memory', 'best']


In [5]:
## 데이터기반의 단어사전 생성 [2] 단어별 정수 부여 
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0


In [6]:
print(word2index)
print(word2index['memory'])

{'for': 1, 'Repeat': 2, 'medicine': 3, 'is': 4, 'the': 5, 'memory': 6, 'best': 7, '<unk>': 0}
6


In [7]:
# 숫자 => 토큰/반환  사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'for', 2: 'Repeat', 3: 'medicine', 4: 'is', 5: 'the', 6: 'memory', 7: 'best', 0: '<unk>'}


In [8]:
print(index2word[2])

Repeat


In [9]:
## 모든 문장 => 수치 벡터화 & 텐서화 진행
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence]     # 각 문자를 정수로 변환

    input_seq,  label_seq = encoded[:-1], encoded[1:]        # 입력 시퀀스와 레이블 시퀀스를 분리
    print(f'{input_seq} => {label_seq}')
    input_seq = torch.LongTensor(input_seq).unsqueeze(0)     # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0)     # 배치 차원 추가
    
    return input_seq, label_seq

In [10]:
X, Y = build_data(sentence, word2index)

[2, 4, 5, 7, 3, 1] => [4, 5, 7, 3, 1, 6]


- 모델 정의 및 설계<hr>

In [11]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        # 워드 임베딩
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, 
                                            embedding_dim=input_size)
        
        # 입력 차원, 은닉 상태의 크기 정의
        self.rnn_layer = nn.RNN(input_size, hidden_size, 
                                batch_first=batch_first)
        
        # 출력 : 원-핫 벡터 크기 또는 단어 집합 크기
        self.linear = nn.Linear(hidden_size, vocab_size) 
        
    def forward(self, x):
        # 1. 임베딩 층
        #    크기 변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        #    크기 변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        #    => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        #    크기 변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        #    크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

- 학습 관련 준비<hr>

In [12]:
# 하이퍼 파라미터
vocab_size  = len(word2index)   # vocab 크기는 임베딩 층, 최종 출력층에 사용
                                # <unk> 토큰을 크기에 포함한다.
input_size  = 5                 # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20                # RNN의 은닉층 크기

In [13]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [14]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)

tensor([[ 0.5833,  0.2771, -0.1314, -0.1018, -0.2362,  0.0480,  0.3824,  0.1373],
        [-0.1944,  0.3531, -0.1316,  0.1183, -0.0499,  0.1751, -0.1727, -0.2192],
        [-0.2632, -0.0055, -0.5205,  0.1289, -0.1377,  0.1138,  0.1352,  0.3230],
        [-0.3129,  0.3078, -0.4745, -0.0596,  0.0105,  0.4253, -0.0415,  0.1842],
        [ 0.1065,  0.1123, -0.5174, -0.1885, -0.1811,  0.2628,  0.1810,  0.4292],
        [ 0.0746,  0.2920, -0.2157, -0.2356, -0.3265,  0.1207,  0.1296,  0.2518]],
       grad_fn=<ViewBackward0>)


In [15]:
print(output.shape)

torch.Size([6, 8])


In [16]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [17]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    print("[{:02d}/201] {:.4f} ".format(step+1, loss))
    
    pred = output.softmax(-1).argmax(-1).tolist()
    print(" ".join(["Repeat"] + decode(pred)))
    print()

[01/201] 2.0561 
Repeat <unk> for best the best for

[02/201] 2.0403 
Repeat <unk> for best the best for

[03/201] 2.0246 
Repeat <unk> for best the best for

[04/201] 2.0091 
Repeat <unk> for best the best for

[05/201] 1.9937 
Repeat <unk> for best the best for

[06/201] 1.9784 
Repeat <unk> for best the best for

[07/201] 1.9632 
Repeat <unk> the best the best best

[08/201] 1.9481 
Repeat <unk> the best the best best

[09/201] 1.9331 
Repeat <unk> the best the best best

[10/201] 1.9181 
Repeat <unk> the best the best best

[11/201] 1.9033 
Repeat <unk> the best the best best

[12/201] 1.8885 
Repeat <unk> the best the best best

[13/201] 1.8737 
Repeat <unk> the best the best best

[14/201] 1.8590 
Repeat <unk> the best the best best

[15/201] 1.8443 
Repeat <unk> the best the best best

[16/201] 1.8296 
Repeat <unk> the best the best best

[17/201] 1.8150 
Repeat <unk> the best the best best

[18/201] 1.8003 
Repeat <unk> the best the best memory

[19/201] 1.7855 
Repeat <unk> th